In [ ]:
import os
import cv2
from os import listdir
from cv_functions import get_image_dimensions, get_d32, resize_save, split_image_in_4

In [ ]:
HOME = os.getcwd()
print(HOME)

In [ ]:
img_path = "output/asphalt_tiles_1600x1200/IMG_9083_1600x1200.jpg"
h, w = get_image_dimensions(image_path=img_path)
# get_d32(h, w, new_width=1600)
print(h, w)

image = cv2.imread(img_path)
cv2.imshow("hello", image)
cv2.waitKey(0)

In [ ]:
import os
from cv_functions import get_image_dimensions, get_d32, resize_save, split_image_in_4

# for every image in input folder, resize and save in output folder.
folder_dir = "datasets_sorted/TiongBahru"
output_dir = "output/tiongbahru_1280_OG"

print("Resizing...")
for image in os.listdir(folder_dir):
    img_path = folder_dir + f"/{image}"
    h, w = get_image_dimensions(img_path)
    if w > h:
        resize_save(img_path, output_dir, 1280, 960)
        # resize_save(img_path, output_dir, 1600, 1200)
    else:
        resize_save(img_path, output_dir, 960, 1280)
        # resize_save(img_path, output_dir, 1200, 1600)
print("Complete.")

In [ ]:
# for every image in input folder, divide into 4 and save in output folder.
folder_dir = "output/asphalt_tiles_1280x960"
output_dir = "output/split"

# image_path = "output/asphalt_tiles_1280x960/IMG_9083_1280x960.jpg"
# split_image_in_4(image_path=image_path, output_folder=output_dir)

print("Splitting...")
for image in os.listdir(folder_dir):
    split_image_in_4(image_path=folder_dir + f"/{image}", output_folder=output_dir)
print("Complete.")

### Increase contrast of image
<a href='https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv'>source</a>
<br>
<a href='https://www.xrite.com/blog/lab-color-space'>read LAB color values</a>

In [ ]:
import cv2
import numpy as np

image_path = 'output/everton_800/IMG_8391_600x800.JPG'

img = cv2.imread(image_path)
# converting to LAB color space
lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l_channel, a, b = cv2.split(lab)

# Applying CLAHE to L-channel
# feel free to try different values for the limit and grid size:
clahe = cv2.createCLAHE(clipLimit=42, tileGridSize=(8,8))
cl = clahe.apply(l_channel)

# merge the CLAHE enhanced L-channel with the a and b channel
limg = cv2.merge((cl,a,b))

# Converting image from LAB Color model to BGR color spcae
enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

new_img = cv2.resize(img, (800, 600))
new_enhanced_img = cv2.resize(enhanced_img, (800, 600))

# Stacking the original image with the enhanced image
result = np.hstack((new_img, new_enhanced_img))
cv2.imshow('Result', result)
cv2.waitKey(0)

#### Increase contrast of one image

In [ ]:
import os
import datetime
from cv_functions import incr_contrast

now = datetime.datetime.now()

# for every image in input folder, increase contrast and save in output folder.
img_path = "datasets_sorted/PunggolMeridian_MRT/IMG_0019.JPG"
output_dir = "output/test"

print("Increasing contrast...")
incr_contrast(image_path=img_path, output_folder=output_dir, clip_limit=5)
    
print("Complete.")

#### Increase contrast of all images in folder

In [1]:
import os
import datetime
from cv_functions import incr_contrast

now = datetime.datetime.now()

# for every image in input folder, increase contrast and save in output folder.
folder_dir = "output/punggol_blk160_1280"
output_dir = "datasets_processed/punggol_blk160_1280"

print("Increasing contrast...")
for image in os.listdir(folder_dir):
    incr_contrast(image_path=folder_dir + f"/{image}", output_folder=output_dir, clip_limit=5)
    
print("Complete.")

Increasing contrast...
Complete.


In [ ]:
from JSON2YOLO.general_json2yolo import convert_coco_json

convert_coco_json(json_dir="../tryout/maskcheck/annotations/instances_default.json")

### Randomly distribute files into train, test, val

#### Distribute images

In [ ]:
import os
import shutil
import random
from pathlib import Path

# src_folder = "output/punggol_blk160_800"
# dst_folder = "transfer_trial"
src_folder = "datasets_processed/punggol_blk160"
dst_folder = "datasets/coco_attempt_2"

train_percent = 0.8
test_percent = 0.1
val_percent = 0.1
random.seed(420)

print("Sorting...")
for dirpath, dirnames, filenames in os.walk(src_folder):
    print(dirpath, dirnames, filenames)

    train_img_path = Path(dst_folder + f"/train/images", exist_ok=True)
    train_lbl_path = Path(dst_folder + f"/train/labels", exist_ok=True)
    test_img_path = Path(dst_folder + f"/test/images", exist_ok=True)
    test_lbl_path = Path(dst_folder + f"/test/labels", exist_ok=True)
    val_img_path = Path(dst_folder + f"/val/images", exist_ok=True)
    val_lbl_path = Path(dst_folder + f"/val/labels", exist_ok=True)

    n = len(filenames)
    index_list = list(range(0, n))
    random.shuffle(index_list)
    train_i_ls = index_list[:int(train_percent*n)]
    test_i_ls = index_list[int(train_percent*n):int((train_percent+test_percent)*n)]
    val_i_ls = index_list[int((train_percent+test_percent)*n):]

    for i, filename in enumerate(filenames):
        src_path = Path(dirpath, filename)
        if i in train_i_ls:
            img_path = train_img_path
            lbl_path = train_lbl_path
        elif i in test_i_ls:
            img_path = test_img_path
            lbl_path = test_lbl_path
        elif i in val_i_ls:
            img_path = val_img_path
            lbl_path = val_lbl_path
        else:
            print("????????????")

        print(f'{i}/{n} - Copying "{src_path}" to "{img_path}".')
        shutil.copy2(src_path, img_path)
    
print("Complete.")

#### Distribute labels corresponding to images

In [ ]:
import os
import shutil
import random
from pathlib import Path

src_folder = "datasets/coco_attempt_2"
label_src = "datasets/coco_attempt_2/annotations/YOLO"

train_img_path = src_folder + "/train/images"
test_img_path = src_folder + "/test/images"
val_img_path = src_folder + "/val/images"

print("Sorting train labels...")
for dirpath, dirnames, filenames in os.walk(train_img_path):
    print(dirpath, dirnames, filenames)

    # destination paths
    train_lbl_path = Path(src_folder + f"/train/labels", exist_ok=True)

    n = len(filenames)

    for i, filename in enumerate(filenames):
        src_path = Path(dirpath, filename)
        label_name = filename[:-4] + ".txt"
        label_path = Path(label_src + f"/{label_name}")
        print(label_path)

        if os.path.exists(label_path):
            print(f'{i}/{n} - Copying "{label_path}" to "{train_lbl_path}".')
            shutil.copy2(label_path, train_lbl_path)

print("Sorting test labels...")
for dirpath, dirnames, filenames in os.walk(test_img_path):
    print(dirpath, dirnames, filenames)

    # destination path
    test_lbl_path = Path(src_folder + f"/test/labels", exist_ok=True)

    n = len(filenames)

    for i, filename in enumerate(filenames):
        src_path = Path(dirpath, filename)
        label_name = filename[:-4] + ".txt"
        label_path = Path(label_src + f"/{label_name}")
        print(label_path)

        if os.path.exists(label_path):
            print(f'{i}/{n} - Copying "{label_path}" to "{test_lbl_path}".')
            shutil.copy2(label_path, test_lbl_path)

print("Sorting val labels...")
for dirpath, dirnames, filenames in os.walk(val_img_path):
    print(dirpath, dirnames, filenames)

    # destination path
    val_lbl_path = Path(src_folder + f"/val/labels", exist_ok=True)

    n = len(filenames)

    for i, filename in enumerate(filenames):
        src_path = Path(dirpath, filename)
        label_name = filename[:-4] + ".txt"
        label_path = Path(label_src + f"/{label_name}")
        print(label_path)

        if os.path.exists(label_path):
            print(f'{i}/{n} - Copying "{label_path}" to "{val_lbl_path}".')
            shutil.copy2(label_path, val_lbl_path)
    
print("Complete.")

### Video processing

#### Extract frames from video

In [1]:
import cv2
from cv_functions import preprocess_frames

preprocess_frames(video_path="datasets_project/Punggol_EdgeField BLK111/GH012496.MP4", output_folder="output/videos/GH012496", frame_count=300)

#### Make images into video

In [1]:
import cv2
import os

source_folder = "output/test_track"

video = cv2.VideoWriter("output/videos/GH012496_tracked2.mp4", 0, 30, (800, 800)) 

for image in os.listdir(source_folder):
    print(image)
    video.write(cv2.imread(source_folder+f"/{image}"))
video.release()